In [125]:
from bs4 import BeautifulSoup as bs
from datetime import datetime as dt
import requests
import re

In [166]:
current_date = dt.now().date()
year = current_date.year
url = 'https://www.bbc.co.uk/weather/2643743'
head = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'en-GB,en;q=0.5',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:128.0) Gecko/20100101 Firefox/128.0'
}

In [129]:
response = requests.get(url, headers=head)

if response.status_code == 200:
    print('Response OK')
    soup = bs(response.text, 'html.parser')
    list_items = soup.select("li[class^='wr-day']")
else:
    print(f'Response failed code {response.status_code}')

Response OK


In [131]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html class="b-reith-sans-font b-pw-1280 no-touch wr-core wr-svg no-js" data-location-id="2643743" data-location-name="London" data-wr-unit--temperature="c" data-wr-unit--windspeed="mph" id="weather-forecast" lang="en">
<head>
<meta content="width=device-width, initial-scale=1, user-scalable=1" name="viewport"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="on" http-equiv="cleartype"/>
<link href="https://ssl.bbc.co.uk/" rel="dns-prefetch"/>
<link href="http://sa.bbc.co.uk/" rel="dns-prefetch"/>
<link href="http://ichef-1.bbci.co.uk/" rel="dns-prefetch"/>
<link href="http://ichef.bbci.co.uk/" rel="dns-prefetch"/>
<style>
        [data-wr-unit--temperature="c"] .wr-c-map__temperature-f,
        [data-wr-unit--temperature="f"] .wr-c-map__temperature-c,
        [data-wr-unit--windspeed="mph"] .wr-c-map__wind-kph,
        [data-wr-unit--windspeed="kph"] .wr-c-map__wind-mph {
            

In [159]:
def retrieve_hourly_data(item):
    return f'Getting hourly data for {item}'

In [175]:
weather_data = []

for item in list_items:
    date = None
    type = None
    low_temps = None
    high_temps = None

    # Date
    title_div = item.find("div", class_="wr-day__title wr-js-day-content-title")
    if title_div:
        date_text = title_div.get("aria-label")
        date_text = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_text)
        date =  dt.strptime(date_text + f' {year}', '%A %d %B %Y').date()

    # If current date, get the 1, 3, 6 and 12 hour data
    if date == current_date:
        print(retrieve_hourly_data(date))

    # Weather type
    type_div = item.find("div", class_="wr-weather-type__text")
    if type_div:
        type = type_div.text
    
    # Low temperature
    low_span = item.find("span", class_='wr-day-temperature__low-value')
    if low_span:
        low_temps = low_span.text

    # High temperature
    high_span = item.find("span", class_='wr-day-temperature__high-value')
    if high_span:
        high_temps = high_span.text


    weather_data.append([date, type, low_temps, high_temps])

print(weather_data)

Date = 2024-08-08
Getting hourly data for 2024-08-08
None
[[datetime.date(2024, 8, 8), 'Thick Cloud', '18° 64°', None], [datetime.date(2024, 8, 9), 'Sunny Intervals', '14° 57°', '25° 77°'], [datetime.date(2024, 8, 10), 'Light Cloud', '15° 59°', '25° 77°'], [datetime.date(2024, 8, 11), 'Sunny', '19° 66°', '29° 84°'], [datetime.date(2024, 8, 12), 'Sunny', '15° 60°', '31° 88°'], [datetime.date(2024, 8, 13), 'Sunny', '17° 62°', '25° 77°'], [datetime.date(2024, 8, 14), 'Sunny Intervals', '13° 55°', '24° 75°'], [datetime.date(2024, 8, 15), 'Drizzle', '13° 55°', '22° 71°'], [datetime.date(2024, 8, 16), 'Light Cloud', '14° 57°', '22° 71°'], [datetime.date(2024, 8, 17), 'Light Cloud', '13° 55°', '22° 71°'], [datetime.date(2024, 8, 18), 'Sunny Intervals', '11° 53°', '21° 70°'], [datetime.date(2024, 8, 19), 'Sunny Intervals', '11° 53°', '21° 71°'], [datetime.date(2024, 8, 20), 'Sunny Intervals', '13° 55°', '21° 71°'], [datetime.date(2024, 8, 21), 'Light Rain', '13° 56°', '21° 71°']]
